In [ ]:
import math

inputdata = {
    "length": 3287,  # total length of panel in mm
    "nspan": 1,      # number of continuous (permanent) spans
    "nprop": 2,      # number of intermediate temporary props (per span during pouring)
    "span": 3287     # slab permanent clear span in mm
}

liveload = {
    "ll": 1.90,   # specified uniform load in kN/m^2
    "wll": 1.90,  # uniform live load per meter strip of slab in kN/m
    "f": 1        # factor for vibration
}

deadload = {
    "dl": 1.52    # superimposed dead load in kN/m^2
}

section = {
    "steelthickness": 0.8,# steel deck thickness in mm
    "fysd": 550,   # steel deck yield stress in MPa
    "slabthickness": 125,   # slab thickness in mm
    "topping": 20, # concrete topping thickness in mm
    "fcprime": 20.7,    # concrete compressive strength in MPa
    "fy": 276,     # steel bar yield strength in MPa
    "botbar": 0,   # bottom bar diameter in mm
    "nbot": 0,     # number of bottom bars per one steel deck panel\
    "d_botbar": 25,# distance of bottom steel bar in mm
    "topbar": 12,  # top bar diameter at near support in mm
    "dprime": 99,  # distance of top bar from bottom of slab (slab - 20 - btop/2) in mm
    "uw": 23.56,   # unit weight of concrete in kN/m^3
    "coef": 0.1,   # moment coefficient at support
    "dww": 10      # diameter of distribution steel in mm
}

asd = 1219 * section ["steelthickness"]/ 0.995 # steel deck cross-sectional area in mm^2
abb = nbot / 0.995 * math.pi * math.pow(botbar,2) / 4 # bottom bar area per meter in mm2
aconc = section ["slabthickness"] * 1000 - 18480 # concrete cross section area per meter in mm^2
es = 200000 # steel elastic modulus in MPa
ec = 4700 * math.sqrt (fcprime) # concrete elastic modulus in MPa
n = es / ec # modular ratio

"""moment of inertia of steel deck per meter"""
if section ["steelthickness"] = 0.8:
    isd = 304581.947
elif section ["steelthickness"]= 1.0:
    isd = 373367.871
elif section ["steelthickness"]= 1.2:
    isd = 442153.796
elif section ["steelthickness"]= 1.6:
    isf = 579725.646
else:
    raise ValueError("Invalid steel deck thickness")

sn = isd / (50-18.333) # section modulus at the top per meter in mm^3
sp = isd / 18.333 # section modulus at the bottom per meter in mm^3
wsd = 77 * asd / 1000000 # weight of steel deck per meter in kN/m
wconc = aconc * uw / 1000000 # weight of concrete per meter in kN/m
wsidl = deadload["dl"] + topping/1000*23.56 # total superimposed dead load per meter in kN/m
wcl = 1.4715 # construction load per meter in kN/m
w1 = 1.6 * wconc + 1.2 * wsd + 1.4 * wcl # in kN/m
wp = 1.6 * liveload ["wll"] + (wsidl + wconc + wsd)


spanprop = inputdata["span"]/ (inputdata ["nprop"] + 1) # span of temporary support in mm

fip1 = 0.125 * w1 * math.pow(spanprop,2) / 0.95 / sn # initial stress at steel deck in MPa using +M = 0.125wl^2
fip2 = 0.08  * w1 * math.pow(spanprop,2) / 0.95 / sn # initial stress at steel deck in MPa using +M = 0.080wl^2


if inputdata ["nprop"] = 0:
    fipa = fip1
else:
    fipa = fip2

if fipa < 0.95*section["fysd"]:
    fip = fipa
else:
    fip = section["fysd"]

"""Check the Stress at Midspan"""

# One meter strip cracked slab section for positive bending

b = 1000 # width in mm
ax = b/(2*n)
bx = asd + abb
cx = asd * (section ["slabthickness"] - 22.149) + abb * (section ["slabthickness"] - section ["d_botbar"])

ac = (-bx+math.pow(math.pow(bx,2)+4*ax*cx),0.5)/(2*ax)
ic = 1000/3/n*math.pow(ac,3)+isd+asd*math.pow(section ["slabthickness"]-ac-22.149,2)+abb*math.pow(section ["slabthickness"]-ac-section["d_botbar"], 2) # cracked moment of inertia per meter in mm^4
sc = ic / (section ["slabthickness"]-ac) # cracked section modulus per meter in mm^3

# One meter strip uncracked slab section for positive bending

b = 1000 # width in mm
b2 = 534

aux = (b-b2)/(2*n)
bux = asd + abb + b2 * section ["slabthickness"] / n
cux = asd * (slabthickness - 22.149) + abb * (slabthickness - c) + b2 * math.pow (slabthicknes,2) / (2*n)

auc = (-bux+math.pow(math.pow(bux,2)+4*aux*cux),0.5)/(2*aux)
iuc = 1000/3/n*math.pow(auc,3)+asd*math.pow(slabthickness-auc-22.149,2)+abb*math.pow(slabthickness-auc-c,2)+534/3/n*math.pow(slabthickness-auc,3) # uncracked moment of inertia per meter in mm^4
suc = iuc / 22.149 # uncracked section modulus per meter in mm^3

# Average Cracked and Uncracked Moment of Inertia

iave = (iuc+ic)/2 # mm4

# Actual Moment at Near Support in kN-m

mp_support = coef*wp*math.pow(inputdata["span"]/1000,2)

# Actual Moment at Midspan in kN-m


if inputdata["nspan"] = 0:
	mp_midspan = 0.125*wp*(inputdata["span"]/1000)^2
else:
	mp_midspan = 0.08*wp*(inputdata["span"]/1000)^2
     
"""Concrete Stress"""

fc =mp_midspan*ac/ic/n*1000000 # actual stress at concrete (transformed) in MPa
allowablefc = 0.85 * fcprime # MPa

concretestress = fc / allowablefc
if concretestress < 1:
    note = "SAFE"
else:
    note = "NOT SAFE"

"""Steel Deck Stress"""

fsd = mp_midspan*1000000*(slabthickness-ac)/ic # actual steel deck stress in MPa
allowablefsd = (0.85*fy - fip) # remaining allowable steel deck stress in MPa

if allowablefsd < 0:
	steeldeckstress = fsd/allowablefsd*(-1)
else:
	steeldeckstress = fsd/allowablefsd
     
if steeldeckstress < 1:
    note = "SAFE"
else:
    note = "NOT SAFE"

"""Steel Bar Stress"""

fs = mp_midspan*1000000*(slabthickness-ac-c)/ic # actual steel bar stress in MPa
allowablefs = 0.9 * fy # allowable steel bar stress in MPa

steelbarstress = fs / allowablefs
if steelbarstress < 1:
    note = "SAFE"
else:
    note = "NOT SAFE"

"""Check Deflection"""
deflection_LL = 0.013* liveload ["wll"]*(inputdata["span"])^4/es/iave # deflection due to live load in mm
allowable_deflection_LL = deflection_LL / 360
ratio_LL = deflection_LL / allowable_deflection_LL
if ratio_LL < 1:
    note = "SAFE"
else:
    note = "NOT SAFE"

deflection = 0.013*( liveload ["wll"]+wsd+wconc+wsidl)*math.pow(inputdata["span"],4)/es/iave #  deflection due to live load plus dead load in mm
allowable_deflection = deflection / 240
ratio = deflection / allowable_deflection
if ratio < 1:
    note = "SAFE"
else:    
    note = "NOT SAFE"

"""Design of Negative Reinforcing"""

as_neg = (-bx_neg-math.pow(math.pow(bx_neg,2)-4*ax_neg*cx_neg),0.5)/(2*ax_neg)
ax_neg = fy/(2*0.85*fcprime*500)
bx_neg = -section["dprime"]
cx_neg = mp_support*1000000/0.9/section["fy"]

# Number of negative bars per meter
if inputdata["nspan"] = 0:
	nbar_neg = 0
else:
    nbar_neg = as_neg / (math.pi * math.pow (section["topbar"], 2) / 4)

# Spacing of negative bars in mm
if nbar_neg = 0:
	sneg = 0
else:
	sneg = 1000/nbar_neg
     
aww = 0.002*section["aconc"] # minimum area for distribution in mm2
nww = aww/(math.pi * math.pow (section["dww"],2) /4) # number of distribution bars per meter
sww = 1000/nww # spacing of distribution bars in mm

## Translate into Class

In [ ]:
# https://atepsmr.tripod.com/dns-deck4.html 
# Steel Deck Reference 
# Steel DN 4 Use in Jollibee Projects 

deck_properties = {
    "gauge_22": {
        "t": 0.8,  # mm
        "w": 8,  # kg/m²
        "as": 1.021,  # mm²/mm
        "I": 518,  # mm⁴/mm
        "sp": 23.4,  # mm³/mm
        "sn": 18.6,  # mm³/mm
        "rb": 11.929,  # kN/m
        "vn": 27.86  # kN/m
    },
    "gauge_20": {
        "t": 1.0,
        "w": 10,
        "as": 1.276,
        "I": 648,
        "sp": 29.2,
        "sn": 23.3,
        "rb": 16.995,
        "vn": 33.76
    },
    "gauge_18": {
        "t": 1.2,
        "w": 12,
        "as": 1.532,
        "I": 777,
        "sp": 35.1,
        "sn": 27.9,
        "rb": 28.779,
        "vn": 44.55
    },
    "gauge_16": {
        "t": 1.6,
        "w": 16,
        "as": 2.042,
        "I": 1037,
        "sp": 46.8,
        "sn": 37.2,
        "rb": 42.778,
        "vn": 55.9
    }
}


span = 3287 #mm
fc = 20.68
fy = 276 
thk = 0.8 
fysd = 550 
span = 3287 #mm
nprop = 2 

ll = 1.9 
sdl = 1.99
wsd = 0.08 
wconc = 2.51 

# load = 1.2*ll + 1.6(sdl + th)

print(deck_properties)

# import math

# class FloorSteelDeckDesign:
#     def __init__(self,span):
#         self.m_to_mm = 1000
        
#         # Constant parameters (to be set by user)
#         self.fc = None              # Concrete compressive strength in MPa
#         self.fy = None              # Yield strength of reinforcement in MPa
#         self.fys = None             # Yield strength of steel deck in MPa
#         self.beta1 = None           # Beta1 factor
#         self.width = None           # Slab width in m
#         self.concreteweight = None  # Concrete weight in kN/m³
        
#         # Live Load parameter in kPa (from data)
#         self.liveloadkpa = None 
        
#         # Dead Load parameters in kPa (from data)
#         self.finishtoppings = None 
#         self.waterproofing = None   
#         self.mechanicalelectrical = None
#         self.ceiling = None
#         self.pefoam = None
#         self.partitions = None
#         self.gypsum = None

#         # Steel Deck and Slab-specific parameters (from data)
#         self.steeldeckthickness = None  # steel deck thickness in mm
#         self.span = span # clear span between supports in m
#         self.assumedslabthickness = None # slab thickness in mm
#         self.tempbar = None # temperature bar diameter in mm
#         self.topbar = None  # top bar diameter in mm

#     def compute_slab_thickness(self):
        
#         recommended = self.span / 33 * self.m_to_mm
#         self.slabthickness = recommended if recommended >= self.assumedslabthickness else self.assumedslabthickness
#         return self.slabthickness

#     def compute_slab_depth(self):
#         self.slabdepth = self.slabthickness - 25.5
#         return self.slabdepth

#     def compute_loads(self):
#         # Keep deadloadkpa in one line
#         slabweight = self.concreteweight * self.clearspan
#         self.deadloadkpa = slabweight + self.finishtoppings + self.waterproofing + self.mechanicalelectrical + self.ceiling + self.pefoam + self.partitions + self.gypsum
#         self.deadload = self.deadloadkpa * self.width  # kN/m
#         self.liveload = self.liveloadkpa * self.width  # kN/m
#         return self.deadload, self.liveload

#     def ultimate_uniform_load(self):
#         if not hasattr(self, "deadload") or not hasattr(self, "liveload"):
#             self.compute_loads()
#         self.ultimateuniformload = 1.2 * self.deadload + 1.6 * self.liveload
#         return self.ultimateuniformload

#     def ultimate_moment(self):
#         self.negativemoment = self.ultimateuniformload * math.pow(self.clearspan, 2) / 11
#         self.positivemoment = self.ultimateuniformload * math.pow(self.clearspan, 2) / 8
#         return self.negativemoment, self.positivemoment

#     def compute_tension_area(self):
#         if self.steeldeckthickness == 0.80:
#             self.tensionarea = 1083
#         elif self.steeldeckthickness == 1.00:
#             self.tensionarea = 1296.1
#         elif self.steeldeckthickness == 1.20:
#             self.tensionarea = 1560.9
#         elif self.steeldeckthickness == 1.40:
#             self.tensionarea = 1828.6
#         elif self.steeldeckthickness == 1.60:
#             self.tensionarea = 2099.6
#         else:
#             raise ValueError("Steel deck thickness value should be 0.80, 1.00, 1.20, 1.40, and 1.60 mm.")
#         return self.tensionarea

#     def check_rho(self):
#         rhobal = self.beta1 * 600 / ( math.pow((self.fys / (0.85 * self.fc)), 1) * (600 + self.fys) )
#         rhomax = 0.75 * rhobal
#         self.rho_act = self.tensionarea / (self.width * self.m_to_mm * self.slabdepth)
#         self.rho = self.rho_act if self.rho_act <= rhomax else rhomax
#         return self.rho

#     def compression_block_depth(self):
#         self.compressionblockdepth = self.rho * (self.fys / (0.85 * self.fc)) * self.slabdepth
#         return self.compressionblockdepth

#     def moment_capacity(self):
#         self.momentcapacity = 0.9 * self.tensionarea * self.fys * (self.slabdepth - self.compressionblockdepth / 2) / 1e6
#         if self.momentcapacity > self.positivemoment:
#             self.momentcapacityresult = "No additional reinforcement is required for bottom pan"
#         else:
#             self.momentcapacityresult = "Additional reinforcement is required for bottom pan"
#         return self.momentcapacityresult, self.momentcapacity

#     def spacing_temperature_bars(self):
#         ast = 0.0018 * self.width * self.m_to_mm * self.slabdepth
#         astemp = math.pi * math.pow(self.tempbar / 2, 2)
#         self.spacingtemperaturebars = astemp / ast * self.width * self.m_to_mm
#         return self.spacingtemperaturebars

#     def compute_Rn(self):
#         self.Rn = self.negativemoment * 1e6 / (0.9 * self.width * self.m_to_mm * math.pow(self.slabthickness - 20 - self.topbar / 2, 2))
#         return self.Rn

#     def top_reinforcement_ratio(self):
#         rhocalc = 0.85 * self.fc / self.fy * (1 - math.sqrt(1 - 2 * self.Rn / (0.85 * self.fc)))
#         rhomin = 1.4 / self.fy
#         self.topreinforcementratio = rhomin if rhocalc <= rhomin else rhocalc
#         return self.topreinforcementratio

#     def spacing_top_bars(self):
#         self.spacingtopbars = (math.pi * math.pow(self.topbar / 2, 2)) / (self.topreinforcementratio * (self.slabthickness - 20 - self.topbar / 2) * self.width * self.m_to_mm) * self.width * self.m_to_mm
#         return self.spacingtopbars

#     def run_design(self):
#         self.compute_slab_thickness()
#         self.compute_slab_depth()
#         self.compute_loads()
#         self.ultimate_uniform_load()
#         self.ultimate_moment()
#         self.compute_tension_area()
#         self.check_rho()
#         self.compression_block_depth()
#         self.moment_capacity()
#         self.spacing_temperature_bars()
#         self.compute_Rn()
#         self.top_reinforcement_ratio()
#         self.spacing_top_bars()
        
#         result = (self.ultimateuniformload, self.negativemoment, self.positivemoment, self.momentcapacity, self.rho, self.momentcapacityresult, self.spacingtemperaturebars, self.topreinforcementratio, self.spacingtopbars)
#         return result

# if __name__ == "__main__":

    # constants = {
    #     "fc": 20.68,             # MPa
    #     "fy": 276,               # MPa
    #     "fys": 276,              # MPa
    #     "beta1": 0.85,
    #     "width": 1,              # m
    #     "concreteweight": 23.60  # kN/m3
    # }


    # data = [
    # # For Slab 1
    #     [0.80, 2.5, 125, 10, 10,                         # slab_params
    #      1.1, 0.05, 0.20, 0.20, 0.18, 1.0, 0.12, 4.8],   # load_params
    # # For Slab 2
    #     [0.80, 2.5, 125, 10, 10,
    #      1.1, 0.05, 0.20, 0.20, 0.18, 1.0, 0.12, 4.8]
    # ]
    #                     #mm                  #m                 #mm             #mm        #mm
    # slab_params = ["steeldeckthickness", "clearspan", "assumedslabthickness", "tempbar", "topbar"]    # all loads in kPa
    # load_params = ["finishtoppings", "waterproofing", "mechanicalelectrical", "ceiling", "pefoam", "partitions", "gypsum", "liveloadkpa"]
    
    # labels = ["Ultimate Uniform Load in kN/m", "Negative Moment in kN-m", "Positive Moment in kN-m", "Moment Capacity in kN-m", "Reinforcement Ratio (rho)", "Moment Capacity Result in kN-m", "Spacing Temperature Bars in mm", "Top Reinforcement Ratio (rho)", "Spacing Top Bars in mm"]

    # for i, row in enumerate(data):
    #     design = FloorSteelDeckDesign()
    #     for attr, value in zip(slab_params, row[:5]):
    #         setattr(design, attr, value)
    #     for attr, value in zip(load_params, row[5:]):
    #         setattr(design, attr, value)
    #     for attr, value in constants.items():
    #         setattr(design, attr, value)

    #     result = design.run_design()
    #     print(f"Slab #{i+1} Results:")
    #     print("-" * 55)
    #     for label, value in zip(labels, result):
    #         print(f"{label}: {value}")
    #     print("-" * 55, "\n")

{'gauge_22': {'t': 0.8, 'w': 8, 'as': 1.021, 'I': 518, 'sp': 23.4, 'sn': 18.6, 'rb': 11.929, 'vn': 27.86}, 'gauge_20': {'t': 1.0, 'w': 10, 'as': 1.276, 'I': 648, 'sp': 29.2, 'sn': 23.3, 'rb': 16.995, 'vn': 33.76}, 'gauge_18': {'t': 1.2, 'w': 12, 'as': 1.532, 'I': 777, 'sp': 35.1, 'sn': 27.9, 'rb': 28.779, 'vn': 44.55}, 'gauge_16': {'t': 1.6, 'w': 16, 'as': 2.042, 'I': 1037, 'sp': 46.8, 'sn': 37.2, 'rb': 42.778, 'vn': 55.9}}


## Reference: Excel

In [ ]:
import math

# --- FUNCTION DEFINITIONS ---

def compute_slabthickness(assumedslabthickness, L, m_to_mm):
    recommendedslabthickness = L / 33 * m_to_mm
    if recommendedslabthickness >= assumedslabthickness:
        slabthickness = recommendedslabthickness
    else:
        slabthickness = assumedslabthickness
    return slabthickness

def compute_slabdepth(slabthickness):
    return slabthickness - 25.5  # mm

def ultimateuniformload(deadload, liveload):
    return 1.2 * deadload + 1.6 * liveload  # kN/m

def ultimatemoment(wu, clearspan):
    neg_mu = wu * math.pow(clearspan, 2) / 11  # kN-m
    pos_mu = wu * math.pow(clearspan, 2) / 8   # kN-m
    return neg_mu, pos_mu

def check_rho(fys, beta1, width, tensionarea, slabdepth, m_to_mm, fc):
    rhobal = beta1 * 600 / ((fys / (0.85 * fc)) * (600 + fys))
    rhomax = 0.75 * rhobal  # TODO: update rhomax if needed
    rhoact = tensionarea / (width * m_to_mm * slabdepth)
    return rhoact if rhoact <= rhomax else rhomax

def depthOfCompressionBlock(rho, slabdepth, fys, fc):
    a = rho * (fys / (0.85 * fc)) * slabdepth  # mm
    return a

def momentcapacity(tensionarea, fys, slabdepth, a, pos_mu):
    # Convert N-mm to kN-m
    mucap = 0.9 * tensionarea * fys * (slabdepth - a / 2) / 1e6
    if mucap > pos_mu:
        result = "No additional reinforcement is required for bottom pan"
    else:
        result = "Additional reinforcement is required for bottom pan"
    return result, mucap

def spacing_temperaturebars(width, m_to_mm, slabdepth, tempbar):
    ast = 0.0018 * width * m_to_mm * slabdepth  # mm²
    astemp = math.pi * math.pow(tempbar / 2, 2)
    spacing = astemp / ast * width * m_to_mm  # mm
    return spacing

def compute_Rn(neg_mu, width, m_to_mm, slabthickness, topbar):
    # 1e6 to convert kN-m to N-mm
    Rn = neg_mu * 1e6 / (0.9 * width * m_to_mm * math.pow(slabthickness - 20 - topbar / 2, 2))
    return Rn

def top_rho(fc, fy, Rn):
    rhocalc = 0.85 * fc / fy * (1 - math.sqrt(1 - 2 * Rn / (0.85 * fc)))
    rhomin = 1.4 / fy
    return rhomin if rhocalc <= rhomin else rhocalc

def spacing_topbars(topbar, rhotop, slabthickness, width, m_to_mm):
    abtop = math.pi * math.pow(topbar / 2, 2)  # mm²
    astop = rhotop * (slabthickness - 20 - topbar / 2) * width * m_to_mm  # mm²
    spacingtop = abtop / astop * width * m_to_mm  # mm
    return spacingtop

# --- MAIN EXECUTION ---
if __name__ == "__main__":
    # DESIGN CRITERIA

    # MATERIAL STRENGTH
    thickness = 1            # steel deck thickness in m
    fc = 21                  # concrete compressive strength in MPa
    fy = 275.8               # minimum yield strength of reinforcing bars in MPa
    fys = 275                # yield strength of steel deck in MPa
    clearspan = 2.5          # clear span between supports in m
    assumedslabthickness = 125  # in mm
    beta1 = 0.85

    # LOADINGS
    width = 1                # in m
    psf_to_kpa = 0.0478802
    slabweight = 61.46 * psf_to_kpa
    finish_toppings = 35 * psf_to_kpa
    ceiling_utilities = 5 * psf_to_kpa
    superimposed_DL = slabweight + finish_toppings + ceiling_utilities
    superimposed_LL = 40 * psf_to_kpa
    deadload = superimposed_DL * width  # kN/m
    liveload = superimposed_LL * width   # kN/m

    # INPUT FROM MANUAL
    tensionarea = 1083     # steel deck tension area in mm²
    tempbar = 10           # diameter of temperature bars in mm
    topbar = 10            # diameter of top bar in mm

    # CONVERSIONS
    m_to_mm = 1000

    # 1. Compute slab thickness and slab depth
    L = clearspan  # For compute_slabthickness, assume L equals clearspan.
    slabthickness = compute_slabthickness(assumedslabthickness, L, m_to_mm)
    print("Slab thickness (mm):", slabthickness)
    
    slabdepth = compute_slabdepth(slabthickness)
    print("Slab depth (mm):", slabdepth)
    
    # 2. Compute ultimate uniform load (wu)
    wu = ultimateuniformload(deadload, liveload)
    print("Ultimate uniform load (kN/m):", wu)
    
    # 3. Compute ultimate moments (negative and positive moments)
    neg_mu, pos_mu = ultimatemoment(wu, clearspan)
    print("Negative moment (kN-m):", neg_mu)
    print("Positive moment (kN-m):", pos_mu)
    
    # 4. Check reinforcement ratio (rho)
    rho = check_rho(fys, beta1, width, tensionarea, slabdepth, m_to_mm, fc)
    print("Reinforcement ratio (rho):", rho)
    
    # 5. Compute depth of compression block (a)
    a = depthOfCompressionBlock(rho, slabdepth, fys, fc)
    print("Depth of compression block (a, mm):", a)
    
    # 6. Compute moment capacity of the steel deck
    result_str, mucap = momentcapacity(tensionarea, fys, slabdepth, a, pos_mu)
    print("Moment capacity (kN-m):", mucap)
    print(result_str)
    
    # 7. Compute spacing for temperature bars
    spacing_temp = spacing_temperaturebars(width, m_to_mm, slabdepth, tempbar)
    print("Spacing for temperature bars (mm):", spacing_temp)
    
    # 8. Calculate top reinforcement parameters:
    Rn = compute_Rn(neg_mu, width, m_to_mm, slabthickness, topbar)
    
    # Compute top reinforcement ratio (rhotop)
    rhotop = top_rho(fc, fy, Rn)
    print("Top reinforcement ratio (rhotop):", rhotop)
    
    # Compute spacing for top bars
    spacing_top = spacing_topbars(topbar, rhotop, slabthickness, width, m_to_mm)
    print("Spacing for top bars (mm):", spacing_top)

## Reference: https://web.facebook.com/EXCELCADSPECIALIST/videos/705677933941066/ - For Checking

In [ ]:
"""
steel deck floor (composite slab) design script
nscp 2015 / aci 318 / aisc 360 / aisi s100 / sdi manuals

this script performs a design check for a steel metal deck with concrete topping
in a composite floor system. the design considers the construction (unshored) stage,
the final composite stage, deflection, web crippling, and shear capacity.
the script uses functions to compute values based on the inputs provided.
"""

import math

def computeUnshoredLoad(slabLoad, deckLoad, imposedLoad, loadFactorSlab=1.6, loadFactorDeck=1.2, loadFactorImposed=1.4):
    """
    compute factored unshored construction loads (kpa)
    includes slab (wet concrete) weight, deck weight, and construction load
    """
    factoredSlabLoad = slabLoad * loadFactorSlab
    factoredDeckLoad = deckLoad * loadFactorDeck
    factoredImposedLoad = imposedLoad * loadFactorImposed
    return factoredSlabLoad + factoredDeckLoad + factoredImposedLoad

def computeShoredLoads(slabLoad, deckLoad, imposedDeadLoad, liveLoad, loadFactorDead=1.2, loadFactorLive=1.6):
    """
    compute final service loads and factored loads for the composite stage.
    returns: serviceDeadLoad, serviceLiveLoad, factoredUniformLoad (kpa)
    """
    deadLoad = slabLoad + deckLoad + imposedDeadLoad
    liveLoadService = liveLoad
    serviceDeadLoad = deadLoad
    serviceLiveLoad = liveLoadService
    factoredUniformLoad = loadFactorDead * deadLoad + loadFactorLive * liveLoadService
    return serviceDeadLoad, serviceLiveLoad, factoredUniformLoad

def computeUnshoredMoment(factoredUniformLoad, clearSpan, stripWidth=1.0):
    """
    compute maximum moment for a single span simply supported deck under uniform load.
    moment = factoredUniformLoad * clearSpan^2 / 8.
    returns moment in kn-m per strip width.
    """
    factoredLoadKnPerM = factoredUniformLoad * stripWidth
    momentKnM = factoredLoadKnPerM * (clearSpan ** 2) / 8.0
    return momentKnM

def computeCompositeMoment(factoredUniformLoad, clearSpan, stripWidth=1.0):
    """
    compute final factored moment for a single span simply supported composite slab.
    moment = factoredUniformLoad * clearSpan^2 / 8.
    returns moment in kn-m.
    """
    factoredLoadKnPerM = factoredUniformLoad * stripWidth
    momentKnM = factoredLoadKnPerM * (clearSpan ** 2) / 8.0
    return momentKnM

def checkMomentCapacity(requiredMoment, capacityMoment):
    """
    check if required moment is less than or equal to capacity moment.
    returns boolean.
    """
    return (requiredMoment <= capacityMoment)

def computeDeflection(serviceLoad, clearSpan, momentInertia, modulusElasticity, stripWidth=1.0):
    """
    compute deflection for a simply supported beam or slab under uniform load.
    using formula: delta = 5 * load * clearSpan^4 / (384 * modulusElasticity * inertia)
    convert clearSpan to mm; momentInertia in mm^4; modulusElasticity in n/mm^2.
    returns deflection in mm.
    """
    loadKnM = serviceLoad * stripWidth   # load in kpa * 1 m = kn/m^2 * m = kn/m
    clearSpanMm = clearSpan * 1000.0
    modulusElasticityNPerMm2 = modulusElasticity * 1.0e6  # mpa to n/mm^2
    # use classic formula; adjust units: load from kn/m to n/m by multiplying by 1000
    deltaMm = (5.0 * loadKnM * 1000.0 * (clearSpanMm ** 4)) / (384.0 * modulusElasticityNPerMm2 * momentInertia)
    return deltaMm

def computeWebCrippling(deckThickness, deckFlangeWidth, deckAngle, bearingLength, yieldStrength, codeCoeffs):
    """
    compute web crippling capacity.
    this is a simplified calculation based on code coefficients.
    codeCoeffs is a dictionary with keys: "c", "cr", "cn", "ch".
    returns interior capacity and exterior capacity in kn.
    """
    coeffC = codeCoeffs.get("c", 10.0)
    coeffCr = codeCoeffs.get("cr", 0.11)
    coeffCn = codeCoeffs.get("cn", 0.21)
    coeffCh = codeCoeffs.get("ch", 0.02)
    thickness = deckThickness
    # approximate angle factor; use sine of deckAngle in radians
    angleFactor = math.sin(math.radians(deckAngle)) ** 0.5 if deckAngle > 0 else 1.0
    interiorCapacity = coeffC * (thickness ** 2) * yieldStrength * angleFactor
    exteriorCapacity = 0.75 * interiorCapacity
    return interiorCapacity, exteriorCapacity

def computeShearCapacityComposite(steelDeckShear, concreteShear):
    """
    compute shear capacity of composite slab.
    returns the sum of steel deck and concrete shear capacities.
    """
    return steelDeckShear + concreteShear

def computeShearStudCapacity(numStuds, singleStudCapacity):
    """
    compute total shear capacity based on number of studs and capacity per stud.
    returns total capacity in kn.
    """
    return numStuds * singleStudCapacity

# ---------------------------------------------------------------------------
# main execution
# ---------------------------------------------------------------------------
if __name__ == "__main__":

    # parameters
    numspan = 3
    clearspan = 3.05          # span in meters
    deckribdepth = 50.0       # deck rib depth in mm
    toppingdepth = 62.5       # concrete topping depth in mm
    deckpitch = 300.0         # deck pitch in mm
    deckgauge = 20            # deck gauge
    deckthickness = 0.91      # deck thickness in mm
    deckweight = 0.07         # deck weight in kpa
    slablWeight = 2.06        # slab (wet concrete) weight in kpa
    imposedDeadLoad = 0.60    # additional dead load (finishes, etc.) in kpa
    liveLoad = 2.90           # live load in kpa
    constructionLoad = 0.96   # construction load (temporary) in kpa

    # material properties
    concreteStrength = 21.0   # mpa, concrete compressive strength
    yieldStrengthDeck = 276.0 # mpa, deck steel yield strength
    modulusConcrete = 21538   # mpa, modulus of elasticity for concrete
    modulusSteel = 200000     # mpa, modulus of elasticity for steel
    modularRatio = modulusSteel / modulusConcrete  # modular ratio

    # geometric properties (from manufacturer data)
    momentInertiaUncracked = 3385323  # mm^4, uncracked section
    momentInertiaCracked = 1576611    # mm^4, cracked section
    effectiveInertia = (momentInertiaUncracked + momentInertiaCracked) / 2.0

    # web crippling constants (from design charts)
    codeCoeffs = {
        "c": 10.0,
        "cr": 0.11,
        "cn": 0.21,
        "ch": 0.02
    }
    deckAngle = 45.0        # degrees
    bearingLength = 50.0    # mm, assumed bearing length

    # -----------------------------------------------------------------------
    # 1. construction stage (unshored) calculations
    # -----------------------------------------------------------------------
    factoredLoadUnshored = computeUnshoredLoad(
        slabLoad=slablWeight,
        deckLoad=deckweight,
        imposedLoad=constructionLoad,
        loadFactorSlab=1.6,
        loadFactorDeck=1.2,
        loadFactorImposed=1.4
    )
    momentUnshored = computeUnshoredMoment(factoredLoadUnshored, clearspan)
    capacityMomentUnshored = 1.30  # kn-m per m width (from manufacturer tables or code)
    constructionStageOk = checkMomentCapacity(momentUnshored, capacityMomentUnshored)

    # -----------------------------------------------------------------------
    # 2. final composite stage calculations
    # -----------------------------------------------------------------------
    serviceDeadLoad, serviceLiveLoad, factoredUniformLoad = computeShoredLoads(
        slabLoad=slablWeight,
        deckLoad=deckweight,
        imposedDeadLoad=imposedDeadLoad,
        liveLoad=liveLoad
    )
    momentFinal = computeCompositeMoment(factoredUniformLoad, clearspan)
    capacityMomentFinal = 5.55  # kn-m (from composite design tables)
    finalCompositeOk = checkMomentCapacity(momentFinal, capacityMomentFinal)

    # -----------------------------------------------------------------------
    # 3. deflection check (service-level)
    # -----------------------------------------------------------------------
    serviceLoadTotal = serviceDeadLoad + serviceLiveLoad  # kpa
    deflectionMm = computeDeflection(
        serviceLoad=serviceLoadTotal,
        clearSpan=clearspan,
        momentInertia=effectiveInertia,
        modulusElasticity=modulusConcrete  # or an effective composite modulus if available
    )
    deflectionLimit180 = (clearspan * 1000.0) / 180.0
    deflectionLimit360 = (clearspan * 1000.0) / 360.0

    # -----------------------------------------------------------------------
    # 4. web crippling check
    # -----------------------------------------------------------------------
    interiorWebCrippling, exteriorWebCrippling = computeWebCrippling(
        deckThickness=deckthickness,
        deckFlangeWidth=150.0,  # assumed value in mm
        deckAngle=deckAngle,
        bearingLength=bearingLength,
        yieldStrength=yieldStrengthDeck,
        codeCoeffs=codeCoeffs
    )
    interiorReaction = 1.17  # kn, assumed reaction from design
    exteriorReaction = 1.10  # kn, assumed reaction from design
    interiorOk = (interiorReaction < interiorWebCrippling)
    exteriorOk = (exteriorReaction < exteriorWebCrippling)

    # -----------------------------------------------------------------------
    # 5. shear capacity check (composite)
    # -----------------------------------------------------------------------
    steelDeckShearCapacity = 14.31  # kn, from steel deck shear capacity data
    concreteShearCapacity = 13.24   # kn, from concrete shear capacity data
    compositeShearCapacity = computeShearCapacityComposite(steelDeckShearCapacity, concreteShearCapacity)
    factoredShearDemand = 3.99  # kn, assumed design shear
    shearOk = (factoredShearDemand <= compositeShearCapacity)

    # -----------------------------------------------------------------------
    # 6. shear stud check (if applicable)
    # -----------------------------------------------------------------------
    singleStudCapacity = 25.0  # kn, capacity of a single shear stud from code or manufacturer data
    numberStuds = 2
    totalStudCapacity = computeShearStudCapacity(numberStuds, singleStudCapacity)

    # -----------------------------------------------------------------------
    # print design results
    # -----------------------------------------------------------------------
    print("======================================================")
    print("steel deck floor (composite slab) design")
    print("number of spans         :", numspan)
    print("clear span (m)          :", clearspan)
    print("deck rib depth (mm)     :", deckribdepth, "   topping depth (mm):", toppingdepth)
    print("------------------------------------------------------")
    print("1) construction stage (unshored)")
    print("   factored load (kpa)      :", factoredLoadUnshored)
    print("   moment (kn-m per m)      :", momentUnshored)
    print("   capacity moment (kn-m)   :", capacityMomentUnshored)
    print("   construction stage ok?   :", "yes" if constructionStageOk else "no")
    print("\n2) final composite stage")
    print("   factored uniform load (kpa)  :", factoredUniformLoad)
    print("   final moment (kn-m)            :", momentFinal)
    print("   capacity moment (kn-m)         :", capacityMomentFinal)
    print("   final composite ok?            :", "yes" if finalCompositeOk else "no")
    print("\n3) deflection check (service-level)")
    print("   service load (kpa)     :", serviceLoadTotal)
    print("   computed deflection (mm):", deflectionMm)
    print("   limit l/180 (mm)       :", deflectionLimit180)
    print("   limit l/360 (mm)       :", deflectionLimit360)
    if deflectionMm < deflectionLimit180:
        print("   deflection is within l/180")
    else:
        print("   deflection exceeds l/180")
    print("\n4) web crippling check")
    print("   interior capacity (kn) :", interiorWebCrippling, ", reaction (kn):", interiorReaction, ", ok? :", interiorOk)
    print("   exterior capacity (kn) :", exteriorWebCrippling, ", reaction (kn):", exteriorReaction, ", ok? :", exteriorOk)
    print("\n5) shear capacity of composite slab")
    print("   steel deck shear capacity (kn) :", steelDeckShearCapacity)
    print("   concrete shear capacity (kn)   :", concreteShearCapacity)
    print("   composite shear capacity (kn)    :", compositeShearCapacity)
    print("   factored shear demand (kn)       :", factoredShearDemand, ", ok? :", shearOk)
    print("\n6) shear stud check (if applicable)")
    print("   single stud capacity (kn) :", singleStudCapacity)
    print("   number of studs           :", numberStuds)
    print("   total stud capacity (kn)  :", totalStudCapacity)
    print("======================================================")
    print("end of design summary")

## Generated by Chatgpt - For Checking

In [ ]:
def checkunshoredspan(deckprofile, thickness, span, wetload, construlive):
    """
    Checks if the chosen deck profile and thickness can support the unshored span.
    
    Parameters:
      deckprofile : str          # deck profile (e.g., "typea", "typeb")
      thickness   : float        # deck sheet thickness in mm (e.g., 1.20)
      span        : float        # unshored span in meters (e.g., 3.2)
      wetload     : float        # wet concrete load in kN/m² (e.g., 2.5)
      construlive : float        # construction live load in kN/m² (e.g., 1.0)
    
    Uses a lookup table (hypothetical data) for maximum span in meters.
    """
    # Example lookup table: key = (deckprofile, thickness)
    # Value = maximum unshored span in meters
    unshoredtable = {
        ("typea", 1.20): 3.0,
        ("typea", 1.40): 3.5,
        ("typeb", 1.20): 3.2,
        ("typeb", 1.60): 4.0
    }
    
    key = (deckprofile, thickness)
    if key not in unshoredtable:
        return (False, "ng: no unshored span data for " + deckprofile + ", " + str(thickness) + " #mm")
    
    maxspan = unshoredtable[key]  # max span in m from table
    # Total construction load (kN/m²) is the sum of wet concrete and live loads.
    totalload = wetload + construlive  # kN/m²
    
    # (In a full design, the load effect would be calculated and compared to moment capacities.)
    if span <= maxspan:
        return (True, "ok: unshored span " + str(span) + " m <= " + str(maxspan) + " m limit for " + deckprofile)
    else:
        return (False, "ng: unshored span " + str(span) + " m exceeds " + str(maxspan) + " m limit for " + deckprofile)

def checkcompositecapacity(deckprofile, thickness, slabthickness, fydeck, superimposed):
    """
    Checks the composite deck-slab bending capacity.
    
    Parameters:
      deckprofile   : str    # deck profile (e.g., "typea", "typeb")
      thickness     : float  # deck sheet thickness in mm (e.g., 1.20)
      slabthickness : float  # total slab thickness in mm (e.g., 120)
      fydeck        : float  # yield strength of deck steel in MPa (e.g., 320)
      superimposed  : float  # superimposed load in kN/m² (e.g., 5.0)
    
    Hypothetical example:
      Base capacity is taken from a table (in kN/m²).
      For each 10 mm above 100 mm slab thickness, add 0.5 kN/m².
      A factor is applied if fydeck > 300 MPa.
    """
    capacitytable = {
        ("typea", 1.20): 8.0,
        ("typea", 1.40): 10.0,
        ("typeb", 1.20): 9.5,
        ("typeb", 1.60): 12.0
    }
    
    key = (deckprofile, thickness)
    if key not in capacitytable:
        return (False, "ng: no capacity data for " + deckprofile + ", " + str(thickness))
    
    basecap = capacitytable[key]  # in kN/m²
    if slabthickness > 100:
        # For each 10 mm above 100 mm, add 0.5 kN/m² capacity
        extrathick = slabthickness - 100  # mm
        addcap = (extrathick // 10) * 0.5  # kN/m²
    else:
        addcap = 0.0
    fyfactor = 1.0
    if fydeck > 300:
        fyfactor = 1.1  # arbitrary bump for higher steel strength
    
    totalcap = (basecap + addcap) * fyfactor  # in kN/m²
    
    if superimposed <= totalcap:
        return (True, "ok: composite capacity ~" + str(round(totalcap,2)) + " kN/m² >= " + str(superimposed) + " kN/m² required")
    else:
        return (False, "ng: demanded " + str(superimposed) + " kN/m² > capacity " + str(round(totalcap,2)) + " kN/m²")

def checkwebcrippling(deckprofile, thickness, bearingwidth, reaction):
    """
    Checks web crippling capacity at supports.
    
    Parameters:
      deckprofile : str   # deck profile (e.g., "typea", "typeb")
      thickness   : float # deck sheet thickness in mm (e.g., 1.20)
      bearingwidth: float # bearing width on support in mm (e.g., 80)
      reaction  : float   # reaction at support in kN per meter (e.g., 45.0)
    
    Uses a simplified lookup table (in kN/m) for base web crippling capacity,
    and applies a bearing factor if the bearing width exceeds 60 mm.
    """
    criptable = {
        ("typea", 1.20): 40,  # base capacity in kN/m
        ("typea", 1.40): 50,
        ("typeb", 1.20): 45,
        ("typeb", 1.60): 60
    }
    
    key = (deckprofile, thickness)
    if key not in criptable:
        return (False, "ng: no web crippling data for " + deckprofile + ", " + str(thickness))
    
    basecrip = criptable[key]  # in kN/m
    bearingfactor = 1.0
    if bearingwidth > 60:
        bearingfactor = 1.1  # if bearing width is greater than 60 mm, capacity increases slightly
    
    capacity = basecrip * bearingfactor  # in kN/m
    
    if reaction <= capacity:
        return (True, "ok: web crippling capacity ~" + str(capacity) + " kN/m >= reaction " + str(reaction) + " kN/m")
    else:
        return (False, "ng: reaction " + str(reaction) + " kN/m > capacity " + str(capacity) + " kN/m")

def checkmaterials(fydeck, fcconc):
    """
    Checks if the material strengths are acceptable.
    
    Parameters:
      fydeck : float   # yield strength of deck steel in MPa (e.g., 320)
      fcconc : float   # concrete compressive strength in MPa (e.g., 27)
    
    Requirements (example):
      fydeck must be at least 250 MPa and fcconc at least 20 MPa.
    """
    if fydeck < 250:
        return (False, "ng: deck steel yield strength (" + str(fydeck) + " MPa) is below minimum 250 MPa")
    if fcconc < 20:
        return (False, "ng: concrete compressive strength (" + str(fcconc) + " MPa) is below 20 MPa")
    return (True, "ok: material strengths are acceptable")

def main():
    print("=== floor deck design (metric units) ===")
    
    # ---------------------------
    # SAMPLE INPUTS (all metric)
    # ---------------------------
    # Material strengths
    fydeck = 320      # MPa (e.g., yield strength of steel deck)
    fcconc = 27       # MPa (e.g., concrete compressive strength)
    
    # Deck geometry
    deckprofile = "typea"  # example deck profile ("typea", "typeb", etc.)
    thickness = 1.20       # mm (deck sheet thickness)
    
    # Slab geometry
    slabthickness = 120    # mm (total composite slab thickness)
    
    # Span and bearing details
    span = 3.2             # m (unshored span)
    bearingwidth = 80      # mm (bearing width at support)
    
    # Loads (in metric units)
    wetload = 2.5          # kN/m² (wet concrete load)
    construlive = 1.0      # kN/m² (construction live load)
    superimposed = 5.0     # kN/m² (superimposed load, e.g., finishes, live load)
    
    # Reaction at support
    reaction = 45.0        # kN/m (reaction from analysis)
    
    # ---------------------------
    # CHECKS
    # ---------------------------
    
    print("\n--- checking material strengths ---")
    matok, matmsg = checkmaterials(fydeck, fcconc)
    print(matmsg)  # e.g., "ok: material strengths are acceptable"
    
    print("\n--- checking unshored span ---")
    spanok, spanmsg = checkunshoredspan(deckprofile, thickness, span, wetload, construlive)
    print(spanmsg)
    
    print("\n--- checking composite capacity ---")
    compok, compmsg = checkcompositecapacity(deckprofile, thickness, slabthickness, fydeck, superimposed)
    print(compmsg)
    
    print("\n--- checking web crippling ---")
    cripok, cripmsg = checkwebcrippling(deckprofile, thickness, bearingwidth, reaction)
    print(cripmsg)
    
    print("\n=== summary of checks ===")
    if matok and spanok and compok and cripok:
        print("all demonstration checks passed. deck selection is acceptable for these sample inputs.")
    else:
        print("one or more checks failed. see details above.")

if __name__ == "__main__":
    main()

In [ ]:
def knton(val):
    # converts kilonewtons to newtons using a multiplier of 1000
    return val * 1000

def checkunshoredspan(deckprofile, gauge, spanft, wetloadpsf, constructionlivepsf):
    """
    Checks if the chosen deck profile and gauge can support the unshored span.
    Loads (in psf) are combined from wet concrete and construction live load.
    Uses a simple lookup table (hypothetical data) for maximum span (ft).
    """
    # Example lookup table: keys are (deckprofile, gauge) with max span in ft
    unshoredtable = {
        ("2wh36", 22): 10,
        ("2wh36", 20): 11,
        ("2wh36", 18): 12,
        ("3wxh36", 22): 11,
        ("3wxh36", 20): 12,
        ("3wxh36", 18): 13
    }
    totalload = wetloadpsf + constructionlivepsf
    key = (deckprofile, gauge)
    if key not in unshoredtable:
        return (False, "no data for " + deckprofile + " gauge " + str(gauge))
    maxspan = unshoredtable[key]
    if spanft <= maxspan:
        return (True, "ok: " + deckprofile + " gauge " + str(gauge) + " can handle " + str(spanft) + " ft unshored span")
    else:
        return (False, "ng: " + deckprofile + " gauge " + str(gauge) + " limited to ~" + str(maxspan) + " ft, but " + str(spanft) + " ft was requested")

def checkcompositecapacity(deckprofile, gauge, slabthicknessin, fydeck, superimposedpsf):
    """
    Checks the composite deck-slab bending capacity using a hypothetical capacity table.
    For each inch above 4 inches slab thickness, adds 10 psf capacity.
    """
    capacitytable = {
        ("2wh36", 22): 60,
        ("2wh36", 20): 80,
        ("2wh36", 18): 100,
        ("3wxh36", 22): 70,
        ("3wxh36", 20): 90,
        ("3wxh36", 18): 120
    }
    key = (deckprofile, gauge)
    if key not in capacitytable:
        return (False, "no composite capacity data for " + deckprofile + " gauge " + str(gauge))
    basecapacity = capacitytable[key]
    if slabthicknessin > 4:
        thicknessfactor = (slabthicknessin - 4) * 10
    else:
        thicknessfactor = 0
    capacitypsf = basecapacity + thicknessfactor
    if superimposedpsf <= capacitypsf:
        return (True, "composite capacity ok: can support " + str(superimposedpsf) + " psf (limit ~" + str(capacitypsf) + " psf)")
    else:
        return (False, "composite capacity ng: demanded " + str(superimposedpsf) + " psf > capacity " + str(capacitypsf) + " psf")

def checkreactionswebcrippling(deckprofile, gauge, bearinglengthin, reactionplf):
    """
    Checks if the deck webs can handle the support reaction without crippling.
    Uses a simplified check with a base capacity (lbs per in) lookup.
    """
    # Convert reaction from lbs/ft to lbs/in; using integer division since no floats are allowed.
    reactionpli = reactionplf // 12
    basecripple = {22: 150, 20: 200, 18: 300, 16: 400}
    if gauge not in basecripple:
        return (False, "no web-cripple data for gauge " + str(gauge))
    capacitypli = basecripple[gauge]
    totalcapacity = capacitypli * bearinglengthin
    required = reactionpli * bearinglengthin
    if required <= totalcapacity:
        return (True, "web crippling ok: capacity ~" + str(totalcapacity) + " lbs >= required ~" + str(required) + " lbs")
    else:
        return (False, "web crippling ng: capacity ~" + str(totalcapacity) + " lbs < required ~" + str(required) + " lbs")

def main():
    print("=== floor deck design checker ===")
    print("sample demonstration of basic checks\n")
    
    # Sample inputs (all values are provided as integers or strings)
    deckprofile = "2wh36"   # example deck profile; acceptable values: "2wh36", "3wxh36"
    gauge = 20              # deck gauge, e.g. 22, 20, 18, or 16
    spanft = 10             # unshored span in feet
    wetloadpsf = 60         # approximate wet concrete load in psf
    constructionlivepsf = 20  # construction live load in psf
    superimposedpsf = 50    # superimposed load in psf
    slabthicknessin = 5     # total slab thickness in inches
    
    # For demonstration purposes, assume:
    fydeck = 33             # yield strength of deck steel (ksi)
    bearinglengthin = 2     # bearing length on support in inches
    reactionplf = 800       # reaction at support in lbs/ft
    
    print("\n--- checking unshored span ---")
    spanok, spanmsg = checkunshoredspan(deckprofile, gauge, spanft, wetloadpsf, constructionlivepsf)
    print(spanmsg)
    
    print("\n--- checking composite capacity ---")
    compok, compmsg = checkcompositecapacity(deckprofile, gauge, slabthicknessin, fydeck, superimposedpsf)
    print(compmsg)
    
    print("\n--- checking web crippling reactions ---")
    webok, webmsg = checkreactionswebcrippling(deckprofile, gauge, bearinglengthin, reactionplf)
    print(webmsg)
    
    print("\n=== summary of checks ===")
    if spanok and compok and webok:
        print("all demonstration checks passed. deck selection looks ok.")
    else:
        print("one or more checks failed or need revision. see messages above.")

if __name__ == "__main__":
    main()